In [26]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

# sklearn config
from sklearn import set_config
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, FunctionTransformer, StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE

#sklearn models
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier

# SƠ LƯỢC DỮ LIỆU

In [27]:
df = pd.read_csv("data/thu_chi_csv.csv", index_col=0)
print(f"Dữ liệu bao gồm {df.shape[0]} dòng và {df.shape[1]} cột:")
print ('Mẫu bảng dữ liệu:\n')
df.head()

Dữ liệu bao gồm 4836 dòng và 10 cột:
Mẫu bảng dữ liệu:



,Ngày,Giờ,Số tiền thu,Số tiền chi,Số dư,Hạng mục cha,Hạng mục con,Chi cho ai,Chuyến đi/Sự kiện,Diễn giải
STT,,,,,,,,,,
1,01/04/2023,16:42,13653000,0,-163716587,Lương,NaN,NaN,NaN,Nam á
2,01/04/2023,16:40,0,300000,-177369587,Hiếu hỉ,Biếu tặng,NaN,NaN,Kem mắt mẹ
3,01/04/2023,16:39,0,37000,-177069587,Ăn uống,Cafe,NaN,NaN,NaN
4,01/04/2023,16:39,0,50000,-177032587,Đi lại,Xăng xe,NaN,NaN,NaN
5,01/04/2023,16:37,0,30000,-176982587,Ăn uống,NaN,NaN,NaN,NaN


In [28]:
df.drop(["Chi cho ai","Chuyến đi/Sự kiện", "Diễn giải"], axis=1, inplace=True)
print ('Mẫu bảng dữ liệu 2:\n')
df.head()

Mẫu bảng dữ liệu 2:



,Ngày,Giờ,Số tiền thu,Số tiền chi,Số dư,Hạng mục cha,Hạng mục con
STT,,,,,,,
1,01/04/2023,16:42,13653000,0,-163716587,Lương,NaN
2,01/04/2023,16:40,0,300000,-177369587,Hiếu hỉ,Biếu tặng
3,01/04/2023,16:39,0,37000,-177069587,Ăn uống,Cafe
4,01/04/2023,16:39,0,50000,-177032587,Đi lại,Xăng xe
5,01/04/2023,16:37,0,30000,-176982587,Ăn uống,NaN


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4836 entries, 1 to 4836
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Ngày          4836 non-null   object
 1   Giờ           4836 non-null   object
 2   Số tiền thu   4836 non-null   int64 
 3   Số tiền chi   4836 non-null   int64 
 4   Số dư         4836 non-null   int64 
 5   Hạng mục cha  4832 non-null   object
 6   Hạng mục con  1732 non-null   object
dtypes: int64(3), object(4)
memory usage: 302.2+ KB


In [30]:
df.describe()

,Số tiền thu,Số tiền chi,Số dư
count,4.836000e+03,4.836000e+03,4.836000e+03
mean,6.763136e+04,9.241239e+04,-1.653381e+08
std,1.400192e+06,5.518006e+05,7.644851e+07
min,0.000000e+00,0.000000e+00,-2.881691e+08
25%,0.000000e+00,1.500000e+04,-2.460990e+08
50%,0.000000e+00,2.500000e+04,-1.669201e+08
75%,0.000000e+00,4.200000e+04,-9.279950e+07
max,5.785000e+07,3.000000e+07,-4.392150e+07


In [31]:
# Convert the date column to a pandas datetime object
df['Ngày'] = pd.to_datetime(df['Ngày'])

# Sort the dataframe by date
df = df.sort_values(by='Ngày').reset_index()
df.drop(["STT"], axis=1, inplace=True)
df.head()

C:\Users\baomc\AppData\Local\Temp\ipykernel_9152\17091528.py:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['Ngày'] = pd.to_datetime(df['Ngày'])


,Ngày,Giờ,Số tiền thu,Số tiền chi,Số dư,Hạng mục cha,Hạng mục con
0,2019-01-01,10:55,0,46000,-43921500,Đi lại,Xăng xe
1,2019-01-01,10:56,0,100000,-44066500,Phát triển bản thân,"Giao lưu, quan hệ"
2,2019-01-01,10:56,0,23000,-44089500,Ăn uống,NaN
3,2019-01-01,18:21,0,20000,-44109500,Ăn uống,NaN
4,2019-01-01,10:55,0,45000,-43966500,Ăn uống,NaN


In [32]:
# Extract year, month, and day columns
df['year'] = df['Ngày'].dt.year
df['month'] = df['Ngày'].dt.month
df['day'] = df['Ngày'].dt.day
df.head()

,Ngày,Giờ,Số tiền thu,Số tiền chi,Số dư,Hạng mục cha,Hạng mục con,year,month,day
0,2019-01-01,10:55,0,46000,-43921500,Đi lại,Xăng xe,2019,1,1
1,2019-01-01,10:56,0,100000,-44066500,Phát triển bản thân,"Giao lưu, quan hệ",2019,1,1
2,2019-01-01,10:56,0,23000,-44089500,Ăn uống,NaN,2019,1,1
3,2019-01-01,18:21,0,20000,-44109500,Ăn uống,NaN,2019,1,1
4,2019-01-01,10:55,0,45000,-43966500,Ăn uống,NaN,2019,1,1


In [37]:
# Extract hour and minute columns
df['hour'] = df['Giờ'].str.split(':', expand=True)[0]
df['minute'] = df['Giờ'].str.split(':', expand=True)[1]

# Convert hour and minute columns to numeric type
df['hour'] = pd.to_numeric(df['hour'])
df['minute'] = pd.to_numeric(df['minute'])
df.head(5)

,Ngày,Giờ,Số tiền thu,Số tiền chi,Số dư,Hạng mục cha,Hạng mục con,year,month,day,hour,minute,hour_rad,minute_rad,hour_sin,hour_cos,minute_sin,minute_cos
0,2019-01-01,10:55,0,46000,-43921500,Đi lại,Xăng xe,2019,1,1,10,55,2.617994,5.759587,5.000000e-01,-8.660254e-01,-0.500000,0.866025
1,2019-01-01,10:56,0,100000,-44066500,Phát triển bản thân,"Giao lưu, quan hệ",2019,1,1,10,56,2.617994,5.864306,5.000000e-01,-8.660254e-01,-0.406737,0.913545
2,2019-01-01,10:56,0,23000,-44089500,Ăn uống,NaN,2019,1,1,10,56,2.617994,5.864306,5.000000e-01,-8.660254e-01,-0.406737,0.913545
3,2019-01-01,18:21,0,20000,-44109500,Ăn uống,NaN,2019,1,1,18,21,4.712389,2.199115,-1.000000e+00,-1.836970e-16,0.809017,-0.587785
4,2019-01-01,10:55,0,45000,-43966500,Ăn uống,NaN,2019,1,1,10,55,2.617994,5.759587,5.000000e-01,-8.660254e-01,-0.500000,0.866025
5,2019-01-03,21:57,0,5000,-49850500,Ăn uống,NaN,2019,1,3,21,57,5.497787,5.969026,-7.071068e-01,7.071068e-01,-0.309017,0.951057
6,2019-01-03,21:46,0,2000,-49845500,Đi lại,Gửi xe,2019,1,3,21,46,5.497787,4.817109,-7.071068e-01,7.071068e-01,-0.994522,0.104528
7,2019-01-03,21:46,0,20000,-49843500,Ăn uống,NaN,2019,1,3,21,46,5.497787,4.817109,-7.071068e-01,7.071068e-01,-0.994522,0.104528
8,2019-01-04,18:02,0,32000,-52139500,Ăn uống,NaN,2019,1,4,18,2,4.712389,0.209440,-1.000000e+00,-1.836970e-16,0.207912,0.978148
9,2019-01-04,12:32,0,9000,-52107500,Ăn uống,Cafe,2019,1,4,12,32,3.141593,3.351032,1.224647e-16,-1.000000e+00,-0.207912,-0.978148


In [34]:
# Convert hour and minute columns to radians
df['hour_rad'] = 2 * np.pi * df['hour'] / 24
df['minute_rad'] = 2 * np.pi * df['minute'] / 60

# Convert hour and minute columns to sin and cos values
df['hour_sin'] = np.sin(df['hour_rad'])
df['hour_cos'] = np.cos(df['hour_rad'])
df['minute_sin'] = np.sin(df['minute_rad'])
df['minute_cos'] = np.cos(df['minute_rad'])

In [35]:
# Aggregate the dataframe by day and sum the money out column for each category
df_money_out = df.groupby(['Ngày', 'Hạng mục cha']).agg({'Số tiền chi': ['sum']})
df_money_out.columns = ['Số_tiền_chi_sum']
df_money_out = df_money_out.reset_index()
df_money_out.head(8)

,Ngày,Hạng mục cha,Số_tiền_chi_sum
0,2019-01-01,Phát triển bản thân,100000
1,2019-01-01,Ăn uống,88000
2,2019-01-01,Đi lại,46000
3,2019-01-03,Ăn uống,25000
4,2019-01-03,Đi lại,2000
5,2019-01-04,Ăn uống,61000
6,2019-01-04,Đi lại,6000
7,2019-01-06,Phát triển bản thân,10000


In [36]:
# Add feature columns for the day of the week and month
df_merged['day_of_week'] = df_merged['date'].dt.dayofweek
df_merged['month'] = df_merged['date'].dt.month

NameError: name 'df_merged' is not defined